AlexNet Model Definition

In [1]:
import tensorflow as tf

def input_placeholder(image_size,image_channel,label_cnt):
    with tf.name_scope("inputlayer"):
        inputs = tf.placeholder("float",[None,image_size,image_size,image_channel],name='inputs')
        labels = tf.placeholder("float",[None,label_cnt],name='labels')
        tf.summary.image("input_image",inputs,max_outputs=4)
    dropout_keep_prob = tf.placeholder("float",None,name='keep_prob')
    learning_rate = tf.placeholder("float",None,name='learning_rate')
    return inputs,labels,dropout_keep_prob,learning_rate
def conv(inputs, kernel_size, output_num, stride_size=1, init_bias=0.0, conv_padding='SAME', stddev=0.01,
         activation_func=tf.nn.relu):
    input_size = inputs.get_shape().as_list()[-1]
    conv_weights = tf.Variable(
        tf.random_normal([kernel_size, kernel_size, input_size, output_num], dtype=tf.float32, stddev=stddev),
        name='conv_weights')
    conv_biases = tf.Variable(tf.constant(init_bias, shape=[output_num], dtype=tf.float32), name='conv_biases')
    conv_layer = tf.nn.conv2d(inputs, conv_weights, [1, stride_size, stride_size, 1], padding=conv_padding)
    conv_layer = tf.nn.bias_add(conv_layer, conv_biases)
    if activation_func:
        conv_layer = activation_func(conv_layer)
    return conv_layer
def fc(inputs, output_size, init_bias=0.0, activation_func=tf.nn.relu, stddev=0.01):
    input_shape = inputs.get_shape().as_list()
    if len(input_shape) == 4:
        fc_weights = tf.Variable(
            tf.random_normal([input_shape[1] * input_shape[2] * input_shape[3], output_size], dtype=tf.float32,
                             stddev=stddev),
            name='fc_weights')
        inputs = tf.reshape(inputs, [-1, fc_weights.get_shape().as_list()[0]])
    else:
        fc_weights = tf.Variable(tf.random_normal([input_shape[-1], output_size], dtype=tf.float32, stddev=stddev),
                                 name='fc_weights')

    fc_biases = tf.Variable(tf.constant(init_bias, shape=[output_size], dtype=tf.float32), name='fc_biases')
    fc_layer = tf.matmul(inputs, fc_weights)
    fc_layer = tf.nn.bias_add(fc_layer, fc_biases)
    if activation_func:
        fc_layer = activation_func(fc_layer)
    return fc_layer
def lrn(inputs, depth_radius=2, alpha=0.0001, beta=0.75, bias=1.0):
    return tf.nn.local_response_normalization(inputs, depth_radius=depth_radius, alpha=alpha, beta=beta, bias=bias)
def inference(inputs,dropout_keep_prob,label_cnt):
    ##todo : change lrn parameters
    #conv layer 1
    with tf.name_scope('conv1layer'):
        conv1 = conv(inputs,7,96,3)
        conv1 = lrn(conv1)
        conv1 = tf.nn.max_pool(conv1,ksize=[1,2,2,1],strides=[1,1,1,1],padding='VALID')
        
    #conv layer 2
    with tf.name_scope('conv2layer'):
        conv2 = conv(conv1,5,256,1,1.0)
        conv2 = lrn(conv2)
        conv2 = tf.nn.max_pool(conv2,ksize=[1,2,2,1],strides=[1,1,1,1],padding='VALID')
    
    #conv layer 3
    with tf.name_scope('conv3layer'):
        conv3 = conv(conv2,2,384,1)
        
    #conv layer 4
    with tf.name_scope('conv4layer'):
        conv4 = conv(conv3,3,384,1,1.0)
        
    #conv layer 5
    with tf.name_scope('conv5layer'):
        conv5 = conv(conv4,3,256,1,1.0)
        conv5 = tf.nn.max_pool(conv5,ksize=[1,3,3,1],strides=[1,2,2,1],padding='VALID')
        
    #fc layer 1
    with tf.name_scope('fc1layer'):
        fc1 = fc(conv5,4096,1.0)
        fc1 = tf.nn.dropout(fc1,dropout_keep_prob)
    
    #fc layer 2
    with tf.name_scope('fc2layer'):
        fc2 = fc(fc1,4096,1.0)
        fc2 = tf.nn.dropout(fc2,dropout_keep_prob)
    
    #fc layer 3 - output
    with tf.name_scope('fc3layer'):
        return fc(fc2,label_cnt,1.0,None)
       
def calc_accuracy(logits,labels):
    #accuracy
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(logits,1),tf.argmax(labels,1)),tf.float32))
        tf.summary.scalar('accuracy',accuracy)
    return accuracy
def calc_loss(logits,labels):
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=labels))
        tf.summary.scalar('loss',loss)
    return loss
def train_rms_prop(loss,learning_rate,decay=0.9,momentum=0.0,epsilon=1e-10,use_locking=False,name='RMSProp'):
    return tf.train.RMSPropOptimizer(learning_rate,decay,momentum,epsilon,use_locking,name).minimize(loss)

Load Data

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import sys
from six.moves import urllib

FLAGS = tf.app.flags.FLAGS
TRAIN_DATA_URL = 'http://file.hovits.com/dl/train.csv'
TEST_DATA_URL = 'http://file.hovits.com/dl/test.csv'

def dense_to_one_hot(labels_dense,num_classes):
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels,num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot
def load_mnist_train(validation_size=2000,batch_size=128):
    download_train()
    data = pd.read_csv(FLAGS.train_path)
    images = data.iloc[:,1:].values
    images = images.astype(np.float)
    images = np.multiply(images,1.0/255.0)
    image_size = images.shape[1]
    image_width = image_height = np.ceil(np.sqrt(image_size)).astype(np.uint8)
    images = images.reshape(-1,image_width,image_height,1)
    labels_flat = data.iloc[:,0].values.ravel()
    labels_count = np.unique(labels_flat).shape[0]
    labels = dense_to_one_hot(labels_flat,labels_count)
    labels = labels.astype(np.uint8)
    validation_images = images[:validation_size]
    validation_labels = labels[:validation_size]
    train_images = images[validation_size:]
    train_labels = labels[validation_size:]
    train_range = list(zip(range(0,len(train_images),batch_size),range(batch_size,len(train_images),batch_size)))
    if len(train_images) % batch_size > 0:
        train_range.append((train_range[-1][1],len(train_images)))
        
    validation_indices = np.arange(len(validation_images))
    return train_images,train_labels,train_range,validation_images,validation_labels,validation_indices
def load_mnist_test(batch_size=128):
    download_test()
    data = pd.read_csv(FLAGS.test_path)
    images = data.iloc[:,1:].values
    images = images.astype(np.float)
    images = np.multiply(images,1.0/255.0)
    image_size = images.shape[1]
    image_width = image_height = np.ceil(np.sqrt(image_size)).astype(np.uint8)
    images = images.reshape(-1,image_width,image_height,1)
    image_account = len(images)
    labels_flat = data.iloc[:,0].values.ravel()
    labels_count = np.unique(labels_flat).shape[0]
    labels = dense_to_one_hot(labels_flat,labels_count)
    labels = labels.astype(np.uint8)
    test_range = list(zip(range(0,image_account,batch_size),range(batch_size,image_account,batch_size)))
    if image_account % batch_size > 0:
        test_range.append((test_range[-1][1],image_account))  
    return images,labels,test_range,image_account
def shuffle_validation(validation_indices,batch_size):
    np.random.shuffle(validation_indices)
    return validation_indices[0:batch_size]
def download_train():
    statinfo = download(FLAGS.train_path,TRAIN_DATA_URL)
    if statinfo:
        print('Training data is successfully downloaded',statinfo.st_size,'bytes.')
    else:
        print('Training data was already downloaded')
def download_test():
    statinfo = download(FLAGS.test_path,TEST_DATA_URL)
    if statinfo:
        print('Test data is successuflly downloaded',statinfo.st_size,'bytes.')
    else:
        print('Test data was already downloaded')
def download(path,url):
    if not os.path.exists(path):
        if not os.path.isdir(os.path.basename(path)):
            os.makedirs(os.path.basename(path))
            
        def _progress(count, block_size, total_size):
            sys.stdout.write(
                '\r>> Downloading %s %.1f%%' % (path, float(count * block_size) / float(total_size) * 100.0))
            sys.stdout.flush()

        file_path, _ = urllib.request.urlretrieve(url, path, _progress)
        print()
        return os.stat(file_path)

Train or Test AlexNet

In [3]:
import tensorflow as tf
import os
import time
import csv

FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_integer('training_epoch',30,"training epoch")
tf.app.flags.DEFINE_integer('batch_size',128,"batch_size")
tf.app.flags.DEFINE_integer('validation_interval',100,"validation interval")
tf.app.flags.DEFINE_float('dropout_keep_prob',0.5,"dropout_keep_prob")
tf.app.flags.DEFINE_float('learning_rate',0.001,'learning rate')
tf.app.flags.DEFINE_float('rms_decay',0.9,"rms optimizer decay")
tf.app.flags.DEFINE_float('weight_decay',0.0005,"L2 regularization weight decay")
tf.app.flags.DEFINE_string('train_path','./data/train.csv',"path to download training data")
tf.app.flags.DEFINE_string('test_path','./data/test.csv',"path to download test data")
tf.app.flags.DEFINE_integer('validation_size',2000,"validation size in training data")
tf.app.flags.DEFINE_string('checkpoint_dir','./checkpoint/',"path to checkpoint file")
tf.app.flags.DEFINE_boolean('is_train',False,"True for train,False for test")
tf.app.flags.DEFINE_string('test_result','./output/test_result.csv',"testing output file path")

image_size = 28
image_channel = 1
label_cnt = 10

def train():
    #build graph
    inputs,labels,dropout_keep_prob,learning_rate = input_placeholder(image_size,image_channel,label_cnt)
    logits = inference(inputs,dropout_keep_prob,label_cnt)
    accuracy = calc_accuracy(logits,labels)
    loss = calc_loss(logits,labels)
    train = tf.train.RMSPropOptimizer(learning_rate,FLAGS.rms_decay).minimize(loss)
    #session
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    #ready for summary
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter('./summary/train',sess.graph)
    validation_writer = tf.summary.FileWriter('./summary/validation')
    #tf saver
    saver = tf.train.Saver()
    if os.path.isfile(FLAGS.checkpoint_name):
        saver.restore(sess,FLAGS.checkpoint_name)
    total_start_time = time.time()
    #load mnist data
    train_images,train_labels,train_range,validation_images,validation_labels,validation_indices = load_mnist_train(
        FLAGS.validation_size,FLAGS.batch_size)
    total_train_len = len(train_images)
    i = 0
    cur_learning_rate = FLAGS.learning_rate
    for epoch in range(FLAGS.training_epoch):
        if epoch % 10 == 0 and epoch > 0:
            cur_learning_rate /= 10
        epoch_start_time = time.time()
        for start,end in train_range:
            batch_start_time = time.time()
            train_x = train_images[start:end]
            train_y = train_labels[start:end]
            if i % 20 == 0:
                summary,_,loss_result = sess.run([merged,train,loss],feed_dict={inputs:train_x,labels:train_y,
                                                                                dropout_keep_prob:FLAGS.dropout_keep_prob,
                                                                                learning_rate:cur_learning_rate})
                train_writer.add_summary(summary,i)
                print('[%s][training][epoch %d,step %d exec %.2f seconds] [file:%5d - %5d / %5d] loss: %3.10f'%(
                time.strftime("%Y-%m-%d %H:%M:%S"),epoch, i,(time.time()-batch_start_time),start,end,total_train_len,loss_result))
            else:
                _,loss_result = sess.run([train,loss],feed_dict={inputs:train_x,labels:train_y,
                                                                dropout_keep_prob:FLAGS.dropout_keep_prob,
                                                                learning_rate:cur_learning_rate})
            
            if i % FLAGS.validation_interval == 0 and i > 0:
                validation_start_time = time.time()
                shuffle_indices = shuffle_validation(validation_indices,FLAGS.batch_size)
                validation_x = validation_images[shuffle_indices]
                validation_y = validation_labels[shuffle_indices]
                summary,accuracy_result,loss_result = sess.run([merged,accuracy,loss],feed_dict={inputs:validation_x,labels:validation_y,
                                                                                                dropout_keep_prob:1.0})
                validation_writer.add_summary(summary,i)
                print('[%s][validation][epoch %d, step %d exec %.2f seconds] accuracy : %1.3f,loss:%3.10f'%(
                    time.strftime("%Y-%m-%d %H:%M:%S"),epoch,i,(time.time()-validation_start_time),accuracy_result,loss_result))
            
            i += 1
        print("[%s][epoch exec %s seconds] epoch : %d"%(time.strftime("%Y-%m-%d %H:%M:%S"),(time.time()-epoch_start_time),epoch))
        saver.save(sess,FLAGS.checkpoint_name),
    print("[%s][total exec %s seconds]"%(time.strftime("%Y-%m-%d %H:%M:%S"),(time.time()-total_start_time)))
    train_writer.close()
    validation_writer.close()
    
def test():
   #build graph
    inputs,labels,dropout_keep_prob,learning_rate = input_placeholder(image_size,image_channel,label_cnt)
    logits = inference(inputs,dropout_keep_prob,label_cnt)
    predict = tf.argmax(logits,1)
    #session
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    
    #tf saver
    saver = tf.train.Saver()
    ckpt = tf.train.get_checkpoint_state(FLAGS.checkpoint_dir)
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
        print("Model restored...")
    #load test data
    test_images,test_labels,test_range,test_account = load_mnist_test(FLAGS.batch_size)
    #ready for result file
    test_result_file = open(FLAGS.test_result,'w')
    csv_writer = csv.writer(test_result_file)
    csv_writer.writerow(['ImageId','Predict','Label'])
    total_start_time = time.time()
    test_total_account = 0
    step = 0
    for file_start,file_end in test_range:
        test_range_account = 0
        test_x = test_images[file_start:file_end]
        test_y = test_labels[file_start:file_end]
        predict_label = sess.run(predict,feed_dict={inputs:test_x,dropout_keep_prob:1.0})
        for i,cur_predict in enumerate(predict_label):
            real_label = np.argmax(test_y[i])
            csv_writer.writerow([step,cur_predict,real_label])
            if cur_predict == real_label:
                test_range_account += 1
            step += 1
            if step % 256 == 0:
                print('[Test file: %4d-%4d,the %d file is predicted %d,the label is %d]'%(file_start,file_end,i,cur_predict,real_label))
                print('Test file: %5d - %5d,Accuracy is:%f'%(file_start,file_end,test_range_account / float(file_end - file_start)))
        test_total_account += test_range_account
    print("[%s][total exec %s second,Accuracy is:%f]"%(time.strftime("%Y-%m-%d %H:%M:%S"),(time.time()-total_start_time),
                                                       test_total_account / float(test_account)))

In [4]:
def main():
    if FLAGS.is_train:
        train()
    else:
        test()
        
main()

INFO:tensorflow:Restoring parameters from ./checkpoint/var.ckpt
Model restored...
Test data was already downloaded
[Test file:  128- 256,the 127 file is predicted 2,the label is 2]
Test file:   128 -   256,Accuracy is:0.992188
[Test file:  384- 512,the 127 file is predicted 2,the label is 2]
Test file:   384 -   512,Accuracy is:1.000000
[Test file:  640- 768,the 127 file is predicted 1,the label is 1]
Test file:   640 -   768,Accuracy is:0.992188
[Test file:  896-1024,the 127 file is predicted 4,the label is 4]
Test file:   896 -  1024,Accuracy is:0.992188
[Test file: 1152-1280,the 127 file is predicted 1,the label is 1]
Test file:  1152 -  1280,Accuracy is:0.992188
[Test file: 1408-1536,the 127 file is predicted 6,the label is 6]
Test file:  1408 -  1536,Accuracy is:1.000000
[Test file: 1664-1792,the 127 file is predicted 7,the label is 7]
Test file:  1664 -  1792,Accuracy is:1.000000
[Test file: 1920-2048,the 127 file is predicted 7,the label is 7]
Test file:  1920 -  2048,Accuracy i

In [9]:
import pandas as pd

input_data = pd.read_csv('./data/train.csv')
input_images = input_data.iloc[:,1:].values
inputlabels_flat = input_data.iloc[:,0].values.ravel()
inputlabels_count = np.unique(inputlabels_flat).shape[0]
print(inputlabels_count)

10
